In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Asthma"
cohort = "GSE185658"

# Input paths
in_trait_dir = "../../input/GEO/Asthma"
in_cohort_dir = "../../input/GEO/Asthma/GSE185658"

# Output paths
out_data_file = "../../output/preprocess/Asthma/GSE185658.csv"
out_gene_data_file = "../../output/preprocess/Asthma/gene_data/GSE185658.csv"
out_clinical_data_file = "../../output/preprocess/Asthma/clinical_data/GSE185658.csv"
json_path = "../../output/preprocess/Asthma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19"
!Series_summary	"Balanced immune responses in airways of patients with asthma are crucial to succesful clearance of viral infection and proper asthma control."
!Series_summary	"We used microarrays to detail the global programme of gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo."
!Series_overall_design	"Bronchial brushings from control individuals and patients with asthma around two weeks before (day -14) and four days after (day 4) experimental in vivo rhinovirus infection were used for RNA isolation and hybrydyzation with Affymetric microarrays."
Sample Characteristics Dictionary:
{0: ['time: DAY14', 'time: DAY4'], 1: ['group: AsthmaHDM', 'group: AsthmaHDMNeg', 'group: Healthy'], 2: ['donor: DJ144', 'donor: DJ113', 'donor: DJ139', 'donor

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on series description, this dataset contains gene expression data from microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait (Asthma) Data
# From sample characteristics, we can see group information in row 1 
# with "AsthmaHDM", "AsthmaHDMNeg", "Healthy" values
trait_row = 1

# Define conversion function for trait
def convert_trait(value):
    # Extract content after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0 for healthy, 1 for asthma)
    if 'Asthma' in value:
        return 1
    elif 'Healthy' in value:
        return 0
    else:
        return None

# 2.2 Age Data - Not provided in the sample characteristics
age_row = None

def convert_age(value):
    # Function defined as required but not used in this dataset
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

# 2.3 Gender Data - Not provided in the sample characteristics
gender_row = None

def convert_gender(value):
    # Function defined as required but not used in this dataset
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value in ['female', 'f', 'woman']:
        return 0
    elif value in ['male', 'm', 'man']:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering on the usability of the dataset
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted data
    print("Preview of extracted clinical data:")
    print(preview_df(selected_clinical_df))
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical data:
{'GSM5621296': [1.0], 'GSM5621297': [1.0], 'GSM5621298': [1.0], 'GSM5621299': [1.0], 'GSM5621300': [1.0], 'GSM5621301': [0.0], 'GSM5621302': [0.0], 'GSM5621303': [1.0], 'GSM5621304': [1.0], 'GSM5621305': [1.0], 'GSM5621306': [1.0], 'GSM5621307': [0.0], 'GSM5621308': [1.0], 'GSM5621309': [1.0], 'GSM5621310': [1.0], 'GSM5621311': [0.0], 'GSM5621312': [1.0], 'GSM5621313': [1.0], 'GSM5621314': [0.0], 'GSM5621315': [1.0], 'GSM5621316': [0.0], 'GSM5621317': [1.0], 'GSM5621318': [1.0], 'GSM5621319': [0.0], 'GSM5621320': [1.0], 'GSM5621321': [0.0], 'GSM5621322': [0.0], 'GSM5621323': [1.0], 'GSM5621324': [1.0], 'GSM5621325': [1.0], 'GSM5621326': [1.0], 'GSM5621327': [1.0], 'GSM5621328': [0.0], 'GSM5621329': [1.0], 'GSM5621330': [1.0], 'GSM5621331': [1.0], 'GSM5621332': [1.0], 'GSM5621333': [1.0], 'GSM5621334': [0.0], 'GSM5621335': [1.0], 'GSM5621336': [0.0], 'GSM5621337': [0.0], 'GSM5621338': [0.0], 'GSM5621339': [1.0], 'GSM5621340': [1.0], 'GSM5621341': [1.0

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Asthma/GSE185658/GSE185658_series_matrix.txt.gz
Gene data shape: (32321, 48)
First 20 gene/probe identifiers:
Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers in the output
# These appear to be probe IDs (numeric identifiers) rather than standard human gene symbols
# Human gene symbols typically follow patterns like BRCA1, TP53, IL6, etc.
# The identifiers shown (7892501, 7892502, etc.) are numeric probe IDs that require mapping to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // ---

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns in gene_annotation that match with gene identifiers in gene_data
# Examine the gene_annotation dataframe columns for probe ID and gene symbol information
print("Gene data index sample:", gene_data.index[:5])
print("\nGene annotation columns:", gene_annotation.columns)

# Based on examining the data:
# - The 'ID' column in gene_annotation contains numeric probe IDs that match gene_data index
# - The 'gene_assignment' column contains gene symbol information

# 2. Get mapping between probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')
print(f"\nMapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"\nGene-level expression data shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:5])

# 4. Save gene expression data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene data index sample: Index(['7892501', '7892502', '7892503', '7892504', '7892505'], dtype='object', name='ID')

Gene annotation columns: Index(['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND',
       'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment',
       'mrna_assignment', 'category'],
      dtype='object')

Mapping dataframe shape: (33297, 2)
First few rows of mapping dataframe:
        ID                                               Gene
0  7896736                                                ---
1  7896738  ENST00000328113 // OR4G2P // olfactory recepto...
2  7896740  NM_001004195 // OR4F4 // olfactory receptor, f...
3  7896742  NR_024437 // LOC728323 // uncharacterized LOC7...
4  7896744  NM_001005221 // OR4F29 // olfactory receptor, ...



Gene-level expression data shape: (117447, 48)
First few gene symbols:
Index(['A-', 'A-3-', 'A-52', 'A-E', 'A-I'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Asthma/gene_data/GSE185658.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Define the correct convert_trait function as established in Step 2
def convert_trait(value):
    # Extract content after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0 for healthy, 1 for asthma)
    if 'Asthma' in value:
        return 1
    elif 'Healthy' in value:
        return 0
    else:
        return None

# Re-extract clinical features using the appropriate conversion functions
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # From step 2
    convert_trait=convert_trait,
    age_row=None,  # No age data available
    convert_age=None,
    gender_row=None,  # No gender data available
    convert_gender=None
)

# Save the processed clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from bronchial brushings comparing healthy individuals with asthma patients before and after rhinovirus infection."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Normalized gene data saved to ../../output/preprocess/Asthma/gene_data/GSE185658.csv
Clinical data saved to ../../output/preprocess/Asthma/clinical_data/GSE185658.csv
Linked data shape: (48, 24222)
Linked data preview (first 5 rows, 5 columns):
            Asthma      A1BG      A1CF       A2M     A2ML1
GSM5621296     1.0  1.064668  0.493903  1.172005  0.959066
GSM5621297     1.0  1.030608  0.476655  0.967298  0.927374
GSM5621298     1.0  1.051556  0.443868  0.933949  0.903052
GSM5621299     1.0  1.109028  0.468362  1.088217  0.954607
GSM5621300     1.0  1.037321  0.460836  0.934322  0.969042


Data shape after handling missing values: (48, 24222)
For the feature 'Asthma', the least common label is '0.0' with 14 occurrences. This represents 29.17% of the dataset.
The distribution of the feature 'Asthma' in this dataset is fine.



Linked data saved to ../../output/preprocess/Asthma/GSE185658.csv
